In [ ]:
import pandas as pd
import numpy as np
import sys
from math import sqrt
sys.path.append('../..')
from modules import utils
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv', usecols=['lat', 'long', 'id'])
kampala_devices.head()

In [ ]:
kampala_df = pd.read_csv('../data/kampala_data.csv', parse_dates=['timestamp'])
kampala_df.head()

In [ ]:
final_df = pd.DataFrame()
# device_ids = []
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value', 'device_number']
# for i, device_id in enumerate(device_ids):
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
#     if len(device_df)!=0:
#         device_ids.append(device_id)
    processed_df = utils.preprocessing(device_df)
#     print(f'{device_id}: {len(processed_df)}')
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

In [ ]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

In [ ]:
final_df = final_df.drop(['device_number'], axis=1)
final_df.head()

In [ ]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
#     X_train, y_train = np.array(X_train), np.array(y_train).reshape(-1, 1)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
#     X_test, y_test = np.array(X_test), np.array(y_test).reshape(-1, 1)
    
    xgb = XGBRegressor(random_state=42)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)
    
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    return rmse
    
#     return mean.numpy(), var.numpy(), Xtest, Ytest, round(rmse, 2)

In [ ]:
len(latitudes)

In [ ]:
rmse_list = []
for i in range(len(latitudes)):
    try:
        rmse = cross_validation(final_df, i)
        rmse_list.append(rmse)
        print(f'Location {i} successful')
    except Exception as e:
        print(e)
        print(f'Location {i} failed')
#         break

mean_rmse = np.mean(rmse_list)
mean_rmse

In [ ]:
rmse_list